In [1]:
import mlflow
import os
from dotenv import load_dotenv
from mlflow.tracking import MlflowClient

# Step 2: Set up the MLflow tracking server
load_dotenv()
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))


In [2]:
mlflow.set_experiment("EXP_2-BoW vs TfIdf")

<Experiment: artifact_location='s3://apoorv-mlfow-bucket/882505162478457184', creation_time=1758359997544, experiment_id='882505162478457184', last_update_time=1758363568812, lifecycle_stage='active', name='EXP_2-BoW vs TfIdf', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [4]:
df = pd.read_csv('D:\datascience\campusx\PROJECTS\Second_project\cleaned_data.csv',index_col=0).dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [5]:
df

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1
...,...,...
37244,jesus,0
37245,kya bhai pure saal chutiya banaya modi aur jab...,1
37246,downvote karna tha par upvote hogaya,0
37247,haha nice,1


In [6]:
# Step 1: Function to run the experiment
def run_experiment(vectorizer_type, ngram_range, vectorizer_max_features, vectorizer_name):
    # Step 2: Vectorization
    if vectorizer_type == "BoW":
        vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)
    else:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)

    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"{vectorizer_name}_{ngram_range}_RandomForest")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with {vectorizer_name}, ngram_range={ngram_range}, max_features={vectorizer_max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", vectorizer_type)
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", vectorizer_max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {vectorizer_name}, {ngram_range}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("D:\datascience\campusx\PROJECTS\Second_project\confusion_matrix.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_{vectorizer_name}_{ngram_range}")

# Step 6: Run experiments for BoW and TF-IDF with different n-grams
ngram_ranges = [(1, 1), (1, 2), (1, 3)]  # unigrams, bigrams, trigrams
max_features = 5000  # Example max feature size

for ngram_range in ngram_ranges:
    # BoW Experiments
    run_experiment("BoW", ngram_range, max_features, vectorizer_name="BoW")

    # TF-IDF Experiments
    run_experiment("TF-IDF", ngram_range, max_features, vectorizer_name="TF-IDF")

2025/09/20 16:07:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/20 16:07:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 1)_RandomForest at: http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/882505162478457184/runs/4311e696ee544bfe9539d13b561d7d12
🧪 View experiment at: http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/882505162478457184


2025/09/20 16:08:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/20 16:09:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 1)_RandomForest at: http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/882505162478457184/runs/5759fafc2ac24a80ac173b199dfa82aa
🧪 View experiment at: http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/882505162478457184


2025/09/20 16:09:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/20 16:10:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TF-IDF_(1, 2)_RandomForest at: http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/882505162478457184/runs/19c62cb41f1a4b7c963a2139a5882a46
🧪 View experiment at: http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/882505162478457184


2025/09/20 16:12:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/20 16:13:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 3)_RandomForest at: http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/882505162478457184/runs/f91ac3c6e5294cfdae10fbd7eb690fb1
🧪 View experiment at: http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/882505162478457184


2025/09/20 16:14:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/20 16:14:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 3)_RandomForest at: http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/882505162478457184/runs/ee89bbb9313d4da48176282e2f34a608
🧪 View experiment at: http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/882505162478457184
